In [1]:
import numpy as np
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix
from collections import defaultdict
from sklearn.metrics import f1_score
import sklearn.preprocessing as pp
from queue import PriorityQueue as pq
#from time import time

In [2]:
PREDICTED_LABELS_FILE \
    = 'predicted_labels2.dat'
TRAIN_FILE \
    = 'train.dat'
TEST_FILE \
    = 'test.dat'
FORMAT_FILE \
    = 'format.dat'

In [3]:
def filterLen(docs, minlen):
    r""" filter out terms that are too short. 
    docs is a list of lists, each inner list is a document represented as a list of words
    minlen is the minimum length of the word to keep
    """
    return [ [t for t in d if len(t) >= minlen ] for d in docs ]

In [4]:
def build_matrix(doc1):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    nrows = len(doc1)
    idx = {}
    tid = 0
    nnz = 0
    for d in doc1:
        nnz += len(set(d))
        for w in d:
            if w not in idx:
                idx[w] = tid
                tid += 1
    ncols = len(idx)
        
    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows + 1, dtype=np.int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in doc1:
        cnt = Counter(d)
        keys = list(k for k,_ in cnt.most_common())
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat


In [5]:
def cosine_sim(training_set,testing_set):
    return training_set * testing_set.T

In [6]:
def get_neighbour_using_knn(l_similarity_matrix_dict_column, l_training_labels, num_of_nearest_neighbors,
                            similarity_threshold):
    q = pq()
    last_key = None
    last_value = None
    for key,value in l_similarity_matrix_dict_column.items():
        last_key = key
        last_value = value
        if value >= similarity_threshold: # checking for min epsilon
            if q.qsize() < num_of_nearest_neighbors:
                q.put((value, key))
            else:
                q.get()
                q.put((value, key))
    neighbours = []
    if not q.empty():
        while not q.empty():
            training_tuple = q.get()
            neighbour = list()
            neighbour.append(l_training_labels[training_tuple[1]])
            neighbour.append(training_tuple[0])
            neighbours.append(neighbour)
    else:
        neighbour = list()
        neighbour.append(l_training_labels[last_key])
        neighbour.append(last_value)
        neighbours.append(neighbour)
    return predict_label(neighbours)

In [7]:
def predict_label(l_neighbours):
    labelVotes = {}
    labelWeightedVotes = {}
    for x in range(len(l_neighbours)):
        label = l_neighbours[x][0]
        if label in labelVotes:
            labelVotes[label] +=1 #majority votes are counted
        else:
            labelVotes[label] = 1
    sorted_votes = sorted(labelVotes, key=labelVotes.__getitem__, reverse=True)    
    return sorted_votes[0][0]

In [8]:
def f1_measure(l_true_labels, l_predicted_labels):
    f1_value = f1_score(l_true_labels, l_predicted_labels, average='weighted')
    return f1_value

In [9]:
def save_predicted_labels(predicted_labels):
    with open(PREDICTED_LABELS_FILE, 'w') as f:
        for item in predicted_labels:
            f.write("%s\n" % item)
        f.close()

In [11]:
def main():
    number_of_neighbors = 20
    neighbor_filter_threshold = 0.2
    # read data from files
    with open(TRAIN_FILE, "r") as file1:
        train_lines = file1.readlines()
    with open(TEST_FILE, "r") as file2:
        test_lines = file2.readlines()
    with open(FORMAT_FILE, "r") as file3:
        format_lines = file3.readlines()

    # store data in list of list(str) for all training set, testing set, true labels and a combined list of list(str) of
    # training and test set
    training_doc = [l.split() for l in train_lines]
    training_labels = []
    for row in training_doc:
        training_labels.append(row[0])
        del row[0]
    training_doc = filterLen(training_doc,3)
    testing_doc = [l.split() for l in test_lines]
    testing_doc = filterLen(testing_doc,3)
    training_testing_doc = training_doc + testing_doc
    true_labels = [l.split() for l in format_lines]

    # building matrix for training and test set with terms from both so that columns are same in both matrix
    training_testing_mat = build_matrix(training_testing_doc)
    training_mat = training_testing_mat[0:len(training_doc)]
    testing_mat = training_testing_mat[len(training_doc):len(training_testing_doc)]

    # normalizing the training and test data set
    training_set = pp.normalize(training_mat.tocsc(), axis=0)
    testing_set = pp.normalize(testing_mat.tocsc(), axis=0)

    # calculate cosine similarities between training and test data set
    similarity_matrix = cosine_sim(training_set, testing_set)

    length_test = len(testing_set.todense())

    predicted_labels = []
    similarity_matrix_dict = {}
    similarity_matrix_dense = similarity_matrix.todense()

    # convert dense matrix to dict
    row_count = np.size(similarity_matrix_dense, 0)
    column_count = np.size(similarity_matrix_dense, 1)
    for column in range(column_count):
        if column % 1000 == 0:
            print('column %d' % column)
        row_dict = {}
        closest_row = 0
        closest_similarity = 0.0
        for row in range(row_count):
            if similarity_matrix_dense[row, column] > neighbor_filter_threshold: #minimum epsilon similarity is checked here
                row_dict[row] = similarity_matrix_dense[row, column]
            if similarity_matrix_dense[row, column] > closest_similarity:
                closest_similarity = similarity_matrix_dense[row, column]
                closest_row = row
        if len(row_dict) == 0: #if there is no neighbour with similarity > minimum similarity then one nearest neighbor is returned
            row_dict[closest_row] = closest_similarity
        similarity_matrix_dict[column] = row_dict

    for x in range(length_test):
        if x % 1000 == 0:
            print('Done with index %d ' % x)
        n = get_neighbour_using_knn(similarity_matrix_dict[x], training_labels, number_of_neighbors, neighbor_filter_threshold)
        predicted_labels.append(n)
    # save predicted labels
    save_predicted_labels(predicted_labels)
    # calculate f-measure score
    f1_val = f1_measure(true_labels, predicted_labels)
    print("f-measure score is %f " % f1_val)


In [ ]:
main()